In [ ]:
import os
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_ollama import OllamaEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_ollama import OllamaLLM
from langchain.chains import RetrievalQA

# 1. 사주 데이터 로드
loader = TextLoader("./data/saju_data.txt", encoding="utf-8")
documents = loader.load()

# 2. 텍스트 조각내기 (의미 단위로 분할)
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
chunks = text_splitter.split_documents(documents)

# 3. Ollama(Gemma) 임베딩 설정
embeddings = OllamaEmbeddings(model="gemma")

# 4. FAISS 벡터 DB 생성 및 로컬 저장
vector_db = FAISS.from_documents(chunks, embeddings)
vector_db.save_local("vector_db/saju_index")

print(f"✅ {len(chunks)}개의 사주 지식을 지식 창고에 저장했습니다!")

In [ ]:
# 1. 모델 선택 (지금은 Gemma 사용)
llm = OllamaLLM(model="gemma")

# 2. 검색기 설정 (가장 관련 있는 지식 3개 찾기)
retriever = vector_db.as_retriever(search_kwargs={"k": 3})

# 3. 질문-답변 체인 생성
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever
)

# 4. 실제 질문 던지기
question = "갑목(甲木) 성격의 리더십 스타일은 어때?"
response = qa_chain.invoke(question)

print("\n[사주 AI의 풀이 결과]")
print(response['result'])